# 1. Introdaction: Project description & work plan

I work as an analyst for the telecom operator Megaline. The company offers its clients two prepaid plans, Surf and Ultimate. The commercial department wants to know which of the plans brings in more revenue in order to adjust the advertising budget.
I am going to carry out a preliminary analysis of the plans based on a relatively small client selection. I will have the data on 500 Megaline clients: who the clients are, where they're from, which plan they use, and the number of calls they made and text messages they sent in 2018. my job is to analyze clients' behavior and determine which prepaid plan brings in more revenue.

# 1.1 Description of the plans


1. **Surf:**
    1. Monthly charge 20 dollars.
    2. 500 monthly minutes, 50 texts, and 15 GB of data.
    3. After exceeding the package limits:
        1. 1 minute: 3 cents
        2. 1 text message: 3 cents
        3. 1 GB of data: $10
        
2. **Ultimate:**
    1. Monthly charge: 70 dollars. 
    2. 3000 monthly minutes, 1000 text messages, and 30 GB of data
    3. After exceeding the package limits:
    4. 1 minute: 1 cent
    5. 1 text message: 1 cent
    6. 1 GB of data: $7

# 1.2 Work plan to completing the project
1. **Step 1: Open the data file and study the general information
File path:**

    1. /datasets/megaline_calls.csv Download dataset
    2. /datasets/megaline_internet.csv Download dataset
    3. /datasets/megaline_messages.csv Download dataset
    4. /datasets/megaline_plans.csv Download dataset
    5. /datasets/megaline_users.csv Download dataset
2. **Step 2. Prepare the data:**
    1. Convert the data to the necessary types
    2. Find and eliminate errors in the data
    3. Explain what errors I found and how I removed them. 
    4. For each user, find:
        1. The number of calls made and minutes used per month
        2. The number of text messages sent per month
        3. The volume of data per month
        4. The monthly revenue from each user
3. **Step 3. Analyze the data:**
    1. Describe the customers' behavior. Find the minutes, texts, and volume of data the users of each plan require per month.
    2. Calculate the mean, dispersion, and standard deviation. Plot histograms. Describe the distributions.
4. **Step 4. Test the hypotheses:**
    1. The average revenue from users of Ultimate and Surf calling plans differs.
    2. The average revenue from users in NY-NJ area is different from that of the users from other regions.
    3. You decide what alpha value to use.
        1. Explain:
        2. How you formulated the null and alternative hypotheses.
        3. What criterion you used to test the hypotheses and why.
5. **Step 5. Write an overall conclusion**

# 1.3 Description of the data

1. **The users table (data on users):**
    1. user_id — unique user identifier
    2. first_name — user's name
    3. last_name — user's last name
    4. age — user's age (years)
    5. reg_date — subscription date (dd, mm, yy)
    6. churn_date — the date the user stopped using the service (if the value is missing, the calling plan was being used when this data was retrieved)
    7. city — user's city of residence
    8. plan — calling plan name
2. **The calls table (data on calls):**
    1. id — unique call identifier
    2. call_date — call date
    3. duration — call duration (in minutes)
    4. user_id — the identifier of the user making the call
3. **The messages table (data on texts):**
    1. id — unique text message identifier
    2. message_date — text message date
    3. user_id — the identifier of the user sending the text
3. **The internet table (data on web sessions):**
    1. id — unique session identifier
    2. mb_used — the volume of data spent during the session (in megabytes)
    3. session_date — web session date
    4. user_id — user identifier
4. **The plans table (data on the plans):**
    1. plan_name — calling plan name
    2. usd_monthly_fee — monthly charge in US dollars
    3. minutes_included — monthly minute allowance
    4. messages_included — monthly text allowance
    5. mb_per_month_included — data volume allowance (in megabytes)
    6. usd_per_minute — price per minute after exceeding the package limits (e.g., if the package includes 100 minutes, the 101st minute will be charged)
    7. usd_per_message — price per text after exceeding the package limits
    8. usd_per_gb — price per extra gigabyte of data after exceeding the package limits (1 GB = 1024 megabytes)

# 2. STEP 1 - Open the data files and study the general information

In [1]:
import pandas as pd
import math
from datetime import datetime
import numpy as np
from scipy import stats as st
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
calls    = pd.read_csv('https://code.s3.yandex.net/datasets/megaline_calls.csv')
internet = pd.read_csv('https://code.s3.yandex.net/datasets/megaline_internet.csv')
message  = pd.read_csv('https://code.s3.yandex.net/datasets/megaline_messages.csv')
plans    = pd.read_csv('https://code.s3.yandex.net/datasets/megaline_plans.csv')
users    = pd.read_csv('https://code.s3.yandex.net/datasets/megaline_users.csv')

In [3]:
for i in [calls]:
    print('Information about the **calls** table')
    print()
    i.info()
    display(i.head(5))
    display(i.tail(5))
    display(i.describe())
    print()
    print('*'*100)

Information about the **calls** table

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
id           137735 non-null object
user_id      137735 non-null int64
call_date    137735 non-null object
duration     137735 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.2+ MB


,id,user_id,call_date,duration
0,1000_93,1000,2018-12-27,8.52
1,1000_145,1000,2018-12-27,13.66
2,1000_247,1000,2018-12-27,14.48
3,1000_309,1000,2018-12-28,5.76
4,1000_380,1000,2018-12-30,4.22


,id,user_id,call_date,duration
137730,1499_199,1499,2018-11-21,8.72
137731,1499_200,1499,2018-10-20,10.89
137732,1499_201,1499,2018-09-21,8.12
137733,1499_202,1499,2018-10-10,0.37
137734,1499_203,1499,2018-12-29,13.86


,user_id,duration
count,137735.000000,137735.000000
mean,1247.658046,6.745927
std,139.416268,5.839241
min,1000.000000,0.000000
25%,1128.000000,1.290000
50%,1247.000000,5.980000
75%,1365.000000,10.690000
max,1499.000000,37.600000



****************************************************************************************************


In [4]:
for i in [internet]:
    print('Information about the **internet** table')
    print()
    i.info()
    display(i.head(5))
    display(i.tail(5))
    display(i.describe())
    print()
    print('*'*100)

Information about the **internet** table

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104825 entries, 0 to 104824
Data columns (total 4 columns):
id              104825 non-null object
user_id         104825 non-null int64
session_date    104825 non-null object
mb_used         104825 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.2+ MB


,id,user_id,session_date,mb_used
0,1000_13,1000,2018-12-29,89.86
1,1000_204,1000,2018-12-31,0.00
2,1000_379,1000,2018-12-28,660.40
3,1000_413,1000,2018-12-26,270.99
4,1000_442,1000,2018-12-27,880.22


,id,user_id,session_date,mb_used
104820,1499_215,1499,2018-10-20,218.06
104821,1499_216,1499,2018-12-30,304.72
104822,1499_217,1499,2018-09-22,292.75
104823,1499_218,1499,2018-12-07,0.00
104824,1499_219,1499,2018-12-24,758.31


,user_id,mb_used
count,104825.000000,104825.000000
mean,1242.496361,366.713701
std,142.053913,277.170542
min,1000.000000,0.000000
25%,1122.000000,136.080000
50%,1236.000000,343.980000
75%,1367.000000,554.610000
max,1499.000000,1693.470000



****************************************************************************************************


In [5]:
for i in [message]:
    print('Information about the **message** table')
    print()
    i.info()
    display(i.head(5))
    display(i.tail(5))
    display(i.describe())
    print()
    print('*'*100)

Information about the **message** table

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
id              76051 non-null object
user_id         76051 non-null int64
message_date    76051 non-null object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


,id,user_id,message_date
0,1000_125,1000,2018-12-27
1,1000_160,1000,2018-12-31
2,1000_223,1000,2018-12-31
3,1000_251,1000,2018-12-27
4,1000_255,1000,2018-12-26


,id,user_id,message_date
76046,1497_526,1497,2018-12-24
76047,1497_536,1497,2018-12-24
76048,1497_547,1497,2018-12-31
76049,1497_558,1497,2018-12-24
76050,1497_613,1497,2018-12-23


,user_id
count,76051.000000
mean,1245.972768
std,139.843635
min,1000.000000
25%,1123.000000
50%,1251.000000
75%,1362.000000
max,1497.000000



****************************************************************************************************


In [6]:
for i in [plans]:
    print('Information about the **plans** table')
    print()
    i.info()
    display(i.head(5))
    display(i.tail(5))
    display(i.describe())
    print()
    print('*'*100)

Information about the **plans** table

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
messages_included        2 non-null int64
mb_per_month_included    2 non-null int64
minutes_included         2 non-null int64
usd_monthly_pay          2 non-null int64
usd_per_gb               2 non-null int64
usd_per_message          2 non-null float64
usd_per_minute           2 non-null float64
plan_name                2 non-null object
dtypes: float64(2), int64(5), object(1)
memory usage: 256.0+ bytes


,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,plan_name
0,50,15360,500,20,10,0.03,0.03,surf
1,1000,30720,3000,70,7,0.01,0.01,ultimate


,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,plan_name
0,50,15360,500,20,10,0.03,0.03,surf
1,1000,30720,3000,70,7,0.01,0.01,ultimate


,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute
count,2.000000,2.000000,2.000000,2.000000,2.00000,2.000000,2.000000
mean,525.000000,23040.000000,1750.000000,45.000000,8.50000,0.020000,0.020000
std,671.751442,10861.160159,1767.766953,35.355339,2.12132,0.014142,0.014142
min,50.000000,15360.000000,500.000000,20.000000,7.00000,0.010000,0.010000
25%,287.500000,19200.000000,1125.000000,32.500000,7.75000,0.015000,0.015000
50%,525.000000,23040.000000,1750.000000,45.000000,8.50000,0.020000,0.020000
75%,762.500000,26880.000000,2375.000000,57.500000,9.25000,0.025000,0.025000
max,1000.000000,30720.000000,3000.000000,70.000000,10.00000,0.030000,0.030000



****************************************************************************************************


In [7]:
for i in [users]:
    print('Information about the **users** table')
    print()
    i.info()
    display(i.head(5))
    display(i.tail(5))
    display(i.describe())
    print()
    print('*'*100)

Information about the **users** table

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
user_id       500 non-null int64
first_name    500 non-null object
last_name     500 non-null object
age           500 non-null int64
city          500 non-null object
reg_date      500 non-null object
plan          500 non-null object
churn_date    34 non-null object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


,user_id,first_name,last_name,age,city,reg_date,plan,churn_date
0,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaN
1,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN
2,1002,Carlee,Hoffman,36,"Las Vegas-Henderson-Paradise, NV MSA",2018-10-21,surf,NaN
3,1003,Reynaldo,Jenkins,52,"Tulsa, OK MSA",2018-01-28,surf,NaN
4,1004,Leonila,Thompson,40,"Seattle-Tacoma-Bellevue, WA MSA",2018-05-23,surf,NaN


,user_id,first_name,last_name,age,city,reg_date,plan,churn_date
495,1495,Fidel,Sharpe,67,"New York-Newark-Jersey City, NY-NJ-PA MSA",2018-09-04,surf,NaN
496,1496,Ariel,Shepherd,49,"New Orleans-Metairie, LA MSA",2018-02-20,surf,NaN
497,1497,Donte,Barrera,49,"Los Angeles-Long Beach-Anaheim, CA MSA",2018-12-10,ultimate,NaN
498,1498,Scot,Williamson,51,"New York-Newark-Jersey City, NY-NJ-PA MSA",2018-02-04,surf,NaN
499,1499,Shena,Dickson,37,"Orlando-Kissimmee-Sanford, FL MSA",2018-05-06,surf,NaN


,user_id,age
count,500.000000,500.000000
mean,1249.500000,45.486000
std,144.481833,16.972269
min,1000.000000,18.000000
25%,1124.750000,30.000000
50%,1249.500000,46.000000
75%,1374.250000,61.000000
max,1499.000000,75.000000



****************************************************************************************************


<div class="alert alert-block alert-success">
<b>Reviewer's comment v1</b>
 
Great start! Now we got a first look at the data and good that you determined next steps.

# 2.1 STEP 1 - SubConclusion about the general information

1. I imported all the required libraries, and loaded all the csv files.
2. I am working with 5 tables:
    1. **calls** - 137735 entries, 4 columns. 
    2. **internet** - 104825 entries, 4 columns.
    3. **message** - 76051 entries, 3 columns.
    4. **plans** - 2 entries, 8 columns.
    5. **users** - 500 entries, 8 columns.
3. I can notice that only the **'churn_date'** column have missing values.
4. I can notice that there is a few columns **all related to date** that the date is object and I need to change the Dtype to daytime so it will be easier to work on.

# 2.2 STEP 2 - Prepare the data:

1. Convert the data to the necessary types
    2. Find and eliminate errors in the data
    3. Explain what errors I found and how I removed them. Note: many calls have a duration of 0.0 minutes. These might be missed calls. Whether or not to preprocess these values is up to you; assess how much their absence would affect the results of your analysis.
    4. For each user, find:
        1. The number of calls made and minutes used per month
        2. The number of text messages sent per month
        3. The volume of data per month
        4. The monthly revenue from each user

**2.2.1 Changing Dtaypes**

In [8]:
calls['call_date'] = pd.to_datetime(calls['call_date'], format='%Y-%m-%d', errors='coerce')
internet['session_date'] = pd.to_datetime(internet['session_date'], format='%Y-%m-%d', errors='coerce')
message['message_date'] = pd.to_datetime(message['message_date'], format='%Y-%m-%d', errors='coerce')
users['reg_date'] = pd.to_datetime(users['reg_date'], format='%Y-%m-%d', errors='coerce')
users['churn_date'] = pd.to_datetime(users['churn_date'], format='%Y-%m-%d', errors='coerce')

**2.2.2 Adding column [month] to the tables**

In [9]:
calls['month'] = calls['call_date'].dt.month
internet['month'] = internet['session_date'].dt.month
message['month'] = message['message_date'].dt.month
users['month'] = users['reg_date'].dt.month
users['churn_month'] = users['churn_date'].dt.month

In [10]:
print (calls['month'].min().max())
print (internet['month'].min().max())
print (message['month'].min().max())
print (users['month'].min().max())
print (users['churn_month'].min().max())

1
1
1
1
7.0


**2.2.3 Rounding data in nessesary [columns] to the tables**

In [11]:
calls['duration'] = np.ceil(calls['duration'])

**2.2.4 Cheacking for duplicates**

In [12]:
calls.duplicated().sum()

0

In [13]:
internet.duplicated().sum()

0

In [14]:
message.duplicated().sum()

0

In [15]:
users.duplicated().sum()

0

**2.2.4 Cheacking for missing values**

In [16]:
calls.isna().sum()

id           0
user_id      0
call_date    0
duration     0
month        0
dtype: int64

In [17]:
internet.isna().sum()

id              0
user_id         0
session_date    0
mb_used         0
month           0
dtype: int64

In [18]:
message.isna().sum()

id              0
user_id         0
message_date    0
month           0
dtype: int64

In [19]:
users.isna().sum()

user_id          0
first_name       0
last_name        0
age              0
city             0
reg_date         0
plan             0
churn_date     466
month            0
churn_month    466
dtype: int64

**When reciving the project I already been told that if there is a Nan in the churn_month column (466 Nan values), its meen that this was the last day of 2018. this is the only Nan value across the tables, and i will fill it with 31-12-2018**

In [20]:
users['churn_date']=users['churn_date'].fillna(pd.to_datetime('2018-12-31'))

In [21]:
users.churn_date.isna().sum()

0

**2.2.5 Cheacking & fixing errors in the data**

In [ ]:
print('Calls error, duration == 0:',calls[calls['duration'] == 0].shape[0])
print('Percentage of rows with Calls error:',\
      round(len(calls[calls['duration']==0])/len(calls)*100, 2),'%')

In [ ]:
print('mb_used error, megabit == 0:',internet[internet['mb_used'] == 0].shape[0])
print('Percentage of rows with mb_used error:',\
      round(len(internet[internet['mb_used']==0])/len(calls)*100, 2),'%')

In [ ]:
calls.drop(calls[calls['duration'] == 0].index,inplace=True)

In [ ]:
internet.drop(internet[internet['mb_used'] == 0].index,inplace=True)

**2.2.5.1 Cheacking the data after the changes**

In [ ]:
calls.info()
print('*'*100)
internet.info()
print('*'*100)
message.info()
print('*'*100)
plans.info()
print('*'*100)
users.info()

**2.2.6 The number of calls made and minutes used per month, per user**

In [ ]:
calls_data = calls.groupby(['user_id', 'month']).duration.agg(['count', 'sum'])
calls_data.columns = ['calls_count', 'calls_duration']
print (calls_data.head(10))
print ()
print (calls_data.describe())

**2.2.7 The number of text messages sent per month**

In [ ]:
message_data = message.groupby(['user_id', 'month'])[['id']].count()
message_data.columns = ['number_of_sms']
print (message_data.head(10))
print ()
print (message_data.describe())

**2.2.8 The volume of data per month**

In [ ]:
internet_use = internet.groupby(['user_id', 'month'])[['mb_used']].sum()
internet_use.columns = ['total_mb_month']
print (internet_use.head(10))
print ()
print (internet_use.describe())

**2.2.8.1 Changing MB per month to GB per month & drop MB column**

In [ ]:
internet_use['total_gb_month'] = (internet_use['total_mb_month'] / 1024).apply(np.ceil)

In [ ]:
internet_use.drop('total_mb_month', axis=1, inplace=True)

In [ ]:
print (internet_use.head(10))
print ()
print (internet_use.describe())

**2.2.9 The monthly plan for each user**

In [ ]:
data_plan = users[['user_id', 'plan']]
print (data_plan.head(10))
print ()
print (data_plan.describe())

**2.2.10 The monthly revenue from each user (work plan):**
1. merge the data from the privious steps to 1 data set.
2. subtract the free package limit from the total number of:
    1. calls.
    2. text messages.
    3. data.
3. multiply the result by the calling plan value.
4. add the monthly charge depending on the calling plan.

**2.2.10.1 Merge the data from [calls_data, message_data, internet_use] for working on 1 dataset in my search for the monthly revenue**

In [ ]:
data_users_revenue = pd.concat([calls_data,message_data,internet_use],axis=1).reset_index().merge(data_plan,on='user_id')
data_users_revenue.columns = ['user_id','month','total_calls','minutes','text_msg','data_used','plan']
print(data_users_revenue.head(10))
print('*'*100)
print (data_users_revenue.info())
print('*'*100)
print (data_users_revenue.describe())

**2.2.10.2 Looking for missing values and fill with 0 - a user may use only text in one month and no calls**

In [ ]:
data_users_revenue.isnull().sum()

In [ ]:
data_users_revenue.fillna(0, inplace=True)

**2.2.10.3 Looking for duplicates in the new table**

In [ ]:
print('data_users_revenue duplicates =', data_users_revenue.duplicated().sum())

**2.2.11 Looking for the values in the plans, before adding a function to extract the exeeded values**

In [ ]:
plans.head(10)

**2.2.12 Function for surf & ultimate plan excess fees**

In [ ]:
def excess_fee(data):
    excess_calls_fee=0
    excess_message_fee=0
    excess_gb_fee=0
    
    if data['plan'] == 'surf':
        excess_calls = 500 - data['minutes']
        if excess_calls < 0:
            excess_calls_fee = (-1 * excess_calls *0.03)  
        excess_message = 50 - data['text_msg']
        if excess_message < 0:
            excess_message_fee = (-1 * excess_message *0.03)  
        excess_gb = 15 - data['data_used']
        if excess_gb < 0:
            excess_gb_fee = (-1 * excess_gb * 10)
        return (excess_calls_fee + excess_message_fee + excess_gb_fee + 20)
    
    else:
        excess_calls = 3000 - data['minutes']
        if excess_calls < 0:
            excess_calls_fee = (-1 * excess_calls *0.01)
        excess_message = 1000 - data['text_msg']
        if excess_message < 0:
            excess_message_fee = (-1 * excess_message *0.01)   
        excess_gb = 30 - data['data_used']
        if excess_gb < 0:
            excess_gb_fee = (-1 * excess_gb * 7)
        return (excess_calls_fee + excess_message_fee + excess_gb_fee + 70)

**2.2.12.1 Apply the Function to the merged dataset & rename the new dataset**

In [ ]:
data_users_revenue['total_fee'] = data_users_revenue.apply(excess_fee, axis=1)

In [ ]:
data_plans_fees = data_users_revenue.sort_values(['user_id', 'month'])

In [ ]:
print(data_plans_fees.head(10))
print('*'*100)
print (data_plans_fees.info())
print('*'*100)
print (data_plans_fees.describe())

**2.2.12.2 Apply plots to the new dataset & research the results**

In [ ]:
fig, ax= plt.subplots(3, figsize=(14,15))

#number of users per plan monthly
user_data=data_plans_fees.groupby(['month','plan'])['user_id'].count().reset_index()
sns.barplot(data=user_data, x='month', y='user_id', hue='plan', palette='bright', ax=ax[0])
ax[0].legend(frameon=True, shadow=True, fontsize=12)
ax[0].set_title('Number of users in each plan', fontsize=14, color='red')

#total revenue from users per plan monthly
sum_data=data_plans_fees.groupby(['month','plan'])['total_fee'].sum().reset_index()
sns.barplot(data=sum_data, x='month', y='total_fee', hue='plan', palette='bright', ax=ax[1])
ax[1].legend(frameon=True, shadow=True, fontsize=12)
ax[1].set_title('Monthly total revenue from users of each plan', fontsize=14, color='red')

# avg revenue from users per plan monthly
mean_data=data_plans_fees.groupby(['month','plan'])['total_fee'].mean().reset_index()
sns.barplot(data=mean_data, x='month', y='total_fee', hue='plan', palette='bright' , ax=ax[2])
ax[2].legend(frameon=True, shadow=True, fontsize=12)
ax[2].set_title('Average monthly revenue from users of each plan', fontsize=14, color='red')

plt.show()

# 2.2.3 STEP 2 - SubConclusion Data preprocessing

1. I changed Dtype where nececssery - **all columns where there is date change to date time to remove error and make the data more usfull for the resarch process.**
2. I checked for duplicates - **there was no duplicates in the data before merge of the tables.**
3. I checked for missing values, values equel to 0 & fix them - there where missing values that I found and probably where miss calls and date related issues.
4. I round the time in nececssery columns.
5. I had column **[month]** to the tables.
6. I checked and fix errors in the data.
7. In the new dataset, I checked again for duplicates and missing values and fix them where necessery.
8. In the **[churn]** column I fill the missing values with the last day of 2018 to avoid deleating almost the entire column.
9. I assum that the missing data in calls and internet where related to **miss calls** and non internet use, I deleted those rows.
10. After dealing with the data in its original datasets, **I merge the 3 new tables in to 1 dataset so I can Analyze the data, Unnecessary columns were dropped.**
11. **I add a function to run trow all the relevent data and add the call,internet use and sms message for each user per month so I can answer the questions on the plan profit.**
12. I used plots to compere the data of each plan according to the user on a monthly bases.
13. observation on the compere data - before complete data analysis:
    1. It was observed that the user number of both plans **increased and in the last month.**
    2. The number of customers who uses the **'surf'** plan was two times more than the number of users who uses the **'ultimate'** plan.
    3. **Even though, 'surf' users pay three times less basic fee per month, the monthly total revenue from the 'surf' plan was almost two times more than 'ultimate'.**
    4. Users of the 'surf' plan more often going over their monthly limit and paying extra money.
        1. This can be clear from the monthly average revenue per user plot.
        2. For example, in the last month, the average revenue from 'surf' plan users was almost equal to 'ultimate' plan users.



# 2.3 STEP 3 - Data analysis:

**2.3.1 Describe the customers' behavior:**

**2.3.1.1 Find the minutes, the users of each plan require per month:**

**2.3.1.1.1 surf:**

In [ ]:
plan_surf = data_plans_fees.query('plan == "surf"')['user_id']
calls_p_plan_surf = data_users_revenue.query('user_id in @plan_surf')
calls_p_plan_surf.info()
calls_p_plan_surf.describe()

**2.3.1.1.2 ultimate:**

In [ ]:
plan_ultimate = data_plans_fees.query('plan == "ultimate"')['user_id']
calls_p_ultimate = data_users_revenue.query('user_id in @plan_ultimate')
calls_p_ultimate.info()
calls_p_ultimate.describe()

**2.3.1.1.3 finding outliers:**

In [ ]:
def outliers_lower_upper(name_df,column):
    q1 = name_df[column].quantile(0.25)
    q3 = name_df[column].quantile(0.75)
    iqr = q3 - q1
    if q1 - iqr*1.5 < 0:
        return (0, q3 + iqr*1.5)
    else:
        return (q1 - iqr*1.5, q3 + iqr*1.5)  

In [ ]:
surf_minutes_outliers      = outliers_lower_upper(calls_p_plan_surf,'minutes')
ultimate_minutes_outliers  = outliers_lower_upper(calls_p_ultimate,'minutes')
print('Outliers limit - plan surf     ',surf_minutes_outliers)
print('Outliers limit - plan ultimate ',ultimate_minutes_outliers)

In [ ]:
len(surf_minutes_outliers)

In [ ]:
len(ultimate_minutes_outliers)

**2.3.1.1.3.1 removing outliers & check the result:**

In [ ]:
surf_minutes_filtered = (calls_p_plan_surf.query('minutes < @surf_minutes_outliers[1]'))
surf_minutes_filtered.info()
surf_minutes_filtered.describe()

In [ ]:
ultimate_minutes_filtered = (calls_p_ultimate.query('minutes < @ultimate_minutes_outliers[1]'))
ultimate_minutes_filtered.info()
ultimate_minutes_filtered.describe()

**2.3.1.1.3.2 showing result trow plots:**

In [ ]:
surf_minutes_filtered['minutes'].hist(density=True, bins=40, figsize = (15, 6))
ultimate_minutes_filtered['minutes'].hist(density=True, bins=40, alpha=0.4)
plt.title('minutes for Surf / ultimate plan users')
labels= ["surf", "ultimate"]
plt.legend(labels)
plt.xlabel('minutes')
plt.ylabel('volume of users')
plt.show()

In [ ]:
surf_minutes_filtered['minutes'].plot(
    kind     = 'hist'
    ,title   = 'minutes used for surf plan users'
    ,figsize = (15, 4)
)
plt.xlabel('minutes')
plt.ylabel('volume of users')
plt.show()

In [ ]:
ultimate_minutes_filtered['minutes'].plot(
    kind     = 'hist'
    ,title   = 'minutes used for ultimate plan users'
    ,figsize = (15, 4)
)
plt.xlabel('minutes')
plt.ylabel('volume of users')
plt.show()

**2.3.1.2 Find the texts, the users of each plan require per month.**

**2.3.1.2.1 finding outliers:**

In [ ]:
surf_text_outliers      = outliers_lower_upper(calls_p_plan_surf,'text_msg')
ultimate_text_outliers  = outliers_lower_upper(calls_p_ultimate,'text_msg')
print('Outliers limit for plan surf     ',surf_text_outliers)
print('Outliers limit for plan ultimate ',ultimate_text_outliers)

In [ ]:
len(surf_text_outliers )

In [ ]:
len(ultimate_text_outliers)

**2.3.1.2.2 remove outliers & check the result:**

In [ ]:
surf_text_filtered = (calls_p_plan_surf.query('text_msg < @surf_text_outliers[1]'))
surf_text_filtered.info()
surf_text_filtered.describe()

In [ ]:
ultimate_text_filtered = (calls_p_ultimate.query('text_msg < @ultimate_text_outliers[1]'))
ultimate_text_filtered.info()
ultimate_text_filtered.describe()

**2.3.1.2.3 showing result trow plots:**

In [ ]:
surf_text_filtered['text_msg'].hist(density=True, bins=20, figsize = (10, 8))
ultimate_text_filtered['text_msg'].hist(density=True, bins=40, alpha=0.4)
plt.title('text for Surf / ultimate plan users')
labels= ["surf", "ultimate"]
plt.legend(labels)
plt.xlabel('text messeges')
plt.ylabel('volume of users')
plt.show()

In [ ]:
surf_text_filtered['text_msg'].plot(
    kind     = 'hist'
    ,title   = 'text used for surf plan users'
    ,figsize = (15, 4)
)
plt.xlabel('text messeges')
plt.ylabel('volume of users')
plt.show()

In [ ]:
ultimate_text_filtered['text_msg'].plot(
    kind     = 'hist'
    ,title   = 'text used for ultimate plan users'
    ,figsize = (15, 4)
)
plt.xlabel('text messeges')
plt.ylabel('volume of users')
plt.show()

**2.3.1.3 Find the volume of data, the users of each plan require per month.** 

**2.3.1.3.1 finding outliers:**

In [ ]:
surf_internet_outliers      = outliers_lower_upper(calls_p_plan_surf,'data_used')
ultimate_internet_outliers  = outliers_lower_upper(calls_p_ultimate,'data_used')
print('Outliers limit for plan surf     ',surf_internet_outliers)
print('Outliers limit for plan ultimate ',ultimate_internet_outliers)

In [ ]:
len(surf_internet_outliers)

In [ ]:
len(ultimate_internet_outliers)

**2.3.1.3.2 remove outliers & check the result:**

In [ ]:
surf_internet_filtered = (calls_p_plan_surf.query('data_used < @surf_internet_outliers[1] and data_used > @surf_internet_outliers[0]'))
surf_internet_filtered.info()
surf_internet_filtered.describe()

In [ ]:
ultimate_internet_filtered = (calls_p_ultimate.query('data_used < @ultimate_internet_outliers[1] & data_used > @ultimate_internet_outliers[0]'))
ultimate_internet_filtered.info()
ultimate_internet_filtered.describe()

**2.3.1.3.3 showing result trow plots:**

In [ ]:
surf_internet_filtered['data_used'].hist(density=True, bins=40, figsize = (15, 6))
ultimate_internet_filtered['data_used'].hist(density=True, bins=40, alpha=0.4)
plt.title('data for Surf / ultimate plan users')
labels= ["surf", "ultimate"]
plt.legend(labels)
plt.xlabel('data usege')
plt.ylabel('volume of users')
plt.show()

In [ ]:
surf_internet_filtered['data_used'].plot(
    kind     = 'hist'
    ,title   = 'internet used for surf plan users'
    ,figsize = (15, 4)
)
plt.xlabel('data usege')
plt.ylabel('volume of users')
plt.show()

In [ ]:
ultimate_internet_filtered['data_used'].plot(
    kind     = 'hist'
    ,title   = 'internet used for ultimate plan users'
    ,figsize = (15, 4)
)
plt.xlabel('data usege')
plt.ylabel('volume of users')
plt.show()

**2.3.1.4 Calculate data values [mean, dispersion, and standard deviation] & Describe the distributions:**

In [ ]:
def two_plan(type,first,second,column):
    data = (
        pd.DataFrame([[type,'surf',
                       first[column].mean(),
                       np.std(first[column]),
                       np.var(first[column])
                      ],
                      [type,'ultimate',
                       second[column].mean(),
                       np.std(second[column]),
                       np.var(second[column])
                      ]], columns = ['type','plan','mean','std','var']))    
    return data   

subject = two_plan('call -  minutes',
                                      surf_minutes_filtered,
                                      ultimate_minutes_filtered,
                                      'minutes')
messages = two_plan('text messege',
                                   surf_text_filtered,
                                   ultimate_text_filtered,
                                   'text_msg')
internet = two_plan('data gb',
                                   surf_internet_filtered,
                                   ultimate_internet_filtered,
                                   'data_used')

subject = subject.append(messages, ignore_index=True)
subject = subject.append(internet, ignore_index=True)
subject = subject.pivot_table(index = ['type','plan'])
subject

**2.3.1.4.1 Calculate data values to help with up coming tasks:**

In [ ]:
data_plans_fees['plan'].value_counts(normalize=True) * 100

In [ ]:
behavior = data_plans_fees.groupby(['plan','month']).agg({'minutes':'sum','text_msg':'sum','data_used':'sum','total_fee':'sum'})
behavior

**2.3.1.4.2 showing and describe the result trow plots:**

In [ ]:
fig, ax= plt.subplots(4, figsize=(19,20))
                                                                                                     
#minutes per plan monthly
minutes_used=data_plans_fees.groupby(['month','plan'])['minutes'].count().reset_index()
sns.barplot(data=minutes_used, x='month', y='minutes', hue='plan', palette= "mako", ax=ax[0])
ax[0].legend(frameon=True, shadow=True, fontsize=12)
ax[0].set_title('Minutes used per plan every month', fontsize=14, color='blue')

#text messege per plan monthly
text_messege=data_plans_fees.groupby(['month','plan'])['text_msg'].sum().reset_index()
sns.barplot(data=text_messege, x='month', y='text_msg', hue='plan', palette="mako", ax=ax[1])
ax[1].legend(frameon=True, shadow=True, fontsize=12)
ax[1].set_title('Text messege used per plan every month', fontsize=14, color='blue')

#internet used per plan monthly
internet_used=data_plans_fees.groupby(['month','plan'])['data_used'].mean().reset_index()
sns.barplot(data=internet_used, x='month', y='data_used', hue='plan', palette="mako" , ax=ax[2])
ax[2].legend(frameon=True, shadow=True, fontsize=12)
ax[2].set_title('Internet used per plan every month', fontsize=14, color='blue')

#total revenue per plan monthly
internet_used=data_plans_fees.groupby(['month','plan'])['total_fee'].mean().reset_index()
sns.barplot(data=internet_used, x='month', y='total_fee', hue='plan', palette="mako" , ax=ax[3])
ax[3].legend(frameon=True, shadow=True, fontsize=12)
ax[3].set_title('Revenue per plan every month', fontsize=14, color='blue')
plt.show()

# 2.3.1 STEP 3 - SubConclusion Data analysis:

1. **The ratio of users of “Surf” and “Ultimate” plans is 70% to 30%.**
2. Users of both plans **spend on average the same number of minutes** , despite the fact that the “Ultimate” plan includes 3000 minutes of calls.
3. Users of the “Surf” plan in average keep within **500 minutes per month**.
4. Surf and Ultimate users send 27 and 36 text messages per month:
    1. Users of the “Surf” plan use **half** all messages included in the plan.
    2. Users of the “Ultimate” plan use less then **5%** of the number of messages included in the plan.
5. On average, users use the same amount of Internet **16 gb:**
    1. Users of the “Surf” plan additionally purchase 1 GB per month.
    2. Users of the “Ultimate” plan, have 30 GB of Internet, but use on average only **half** of the GB whitin the plan.
6. Distribution:
    1. "Calls" and "Internet" histograms have **"Normal Distribution"** type.
    2. "Message" histograms have **"Right Skewed Distribution"** type.
7. The histogram almost have **similar distribution between surf and ultimate.**
8. Surf plan brings **more** total revenue than ultimate plan since surf have **more users than ultimate.**
9. Not all users have 12 active months.
10. Users most often exceed the limits of packages for the service **"internet".**

# Answer to Reviewer's comment v1:
1. **I most defenatly will had recomendations and improve all the first 3 projects base on the new learnings/studying and reviewers recomendations. I plan on doing all the improvments in between moduls when we have our break in the studying. After that I will have nice 3 projects to had to my projects file. thank you for spending the time to give me more tips on how to improve my work.**

# 2.4 STEP 4 - Test the hypotheses:

1. The average revenue from users of Ultimate and Surf calling plans differs.
2. The average revenue from users in NY-NJ area is different from that of the users from other regions.
    1. You decide what alpha value to use.
3. Explain:
    1. How you formulated the null and alternative hypotheses.
    2. What criterion you used to test the hypotheses and why.

**2.4.1 Test hypotheses no 1 -  The average revenue from users of Ultimate and Surf calling plans differs:**
1. **[null hypothesis and an alternative hypothesis]:**
    1. **H0: The average profit of Ultimate == the average profit of Surf.**
    2. **H1: The average profit of Ultimate and Surf calling plans == not equel.**

In [ ]:
filter_profit = st.zscore(data_plans_fees['total_fee'])
abs_filter_profit = np.abs(filter_profit)
filtered_entries = (abs_filter_profit < 3)
data_plans_fees = data_plans_fees[filtered_entries]


# surf users
surfplan = data_plans_fees.query('plan == "surf"')
surf_plan_Average_Profit = surfplan['total_fee'].mean().round(decimals=2)
print('The average monthly profit from SURF users is: $' + str(surf_plan_Average_Profit))

# ultimate users
ultimateplan = data_plans_fees.query('plan == "ultimate"')
ultimate_plan_Average_Profit = ultimateplan['total_fee'].mean().round(decimals=2)
print('The average monthly profit from ULTIMATE users is: $' + str(ultimate_plan_Average_Profit))

#  t-test
results = st.ttest_ind(surfplan['total_fee'], ultimateplan['total_fee'], equal_var=False)
p_value = results.pvalue
alpha = 0.05

print ('p_value =',p_value)

if p_value < alpha:
    print('Reject H0')
else:
    print('Cannot reject H0')

**2.4.2 Test hypotheses no 2 - The average revenue from users in NY-NJ area is different from that of the users from other regions.**
1. **[null hypothesis and an alternative hypothesis]:**
    1. **H0: The average profit in NY-NJ area == the average profit in other regions.**
    2. **H1: The average profit from the users in NY-NJ area is different from that of the users from other regions.**

In [ ]:
# users in NY-NJ area
NY_NJ = users[users['city'].str.contains('NY-NJ')]
users_from_nynj = NY_NJ['user_id']
NYNJ_Average = data_plans_fees.query('user_id in @users_from_nynj')
NYNJ_Average_Profit = NYNJ_Average['total_fee'].mean().round(decimals=2)
print('The average monthly profit IN NY-NJ area is: $' + str(NYNJ_Average_Profit))

# users not in NY-NJ area
ex_NYNJ= users[~users['city'].str.contains('NY-NJ')]
users_not_from_nynj = ex_NYNJ['user_id']
Not_NYNJ_Average = data_plans_fees.query('user_id in @users_not_from_nynj')
Not_NYNJ_Average_Profit = Not_NYNJ_Average['total_fee'].mean().round(decimals=2)
print('The average monthly profit NOT in NY-NJ area is: $' + str(Not_NYNJ_Average_Profit))
#  t-test
results = st.ttest_ind(NYNJ_Average['total_fee'], Not_NYNJ_Average['total_fee'], equal_var=False)

p_value = results.pvalue

alpha = 0.05

print ('p_value =',p_value)

if p_value < alpha:
    print('Reject H0')
else:
    print('Cannot reject H0')

# 2.4 STEP 4 - SubConclusion hypotheses:

1. Hypotheses no 1 - The average revenue from users of Ultimate and Surf calling plans differs:
    * **The revenues from each plan are different, so I reject the Hypotheses.**
    * I used 0.05 as alpha, that give me 95% confidence that revenue coming from each plan is not equal.
    * The average monthly profit from SURF users is: (USD54.76).
    * The average monthly profit from ULTIMATE users is: (USD72.33).
    * As I learn in the privious tasks, The ratio of users of Surf and Ultimate plans is 70% to 30%.
2. Hypotheses no 2 - The average revenue from users in NY-NJ area is different from that of the users from other regions:
    1. **The revenues from diffrent area & NY-NJ are not significantly different, so I can't reject the Hypotheses.**
    2. I used 0.05 as alpha, that give me 95% confidence that revenue coming from each area is not diffrent.
    2. The average monthly profit IN NY-NJ area is: (USD57.22).
    3. The average monthly profit NOT in NY-NJ area is: (USD61.01).
    

# 2.5 STEP 5 - overall conclusion:

1. **Step 1+2 conclusion:**
    1. **I recived 5 datasets:**
        1. **Calls** - Store the data of every call.
        2. **Internet** - Store the data of internet session.
        3. **Messages** -  Store the data of messages
        4. **Users** -  Store the data of each user
        5. **Plan** - Store the data of the plan being used.
    2. **There where servel errors & values needed attention in the data:**
        1. I converted date in the data sets from object type to datetime data type.
        2. The missing values in the churn_date column were filled with the last day of the data - 2018-12-31.
        3. Around 20% of individual call durations and 10% of internet sessions (mb_used) hava a value of 0, These zero values were droped.
        4. I rounded up values where necessery.
        5. I merge the 3 new tables in to 1 dataset so I can Analyze the data, Unnecessary columns were dropped.
2. **Step 3 conclusion:**
    1. Several plots were made to get an insight into revenue from each plan and users.
    2. It was observed that the user number of both plans increased and in the last month.
    3. The ratio of users of Surf and Ultimate plans is **70% to 30%.**
    4. 'surf' users pay three times less basic fee per month.
    5. The monthly total revenue from the **'surf'** plan was almost two times more than **'ultimate'.**
    6. Users who use the 'surf' plan more often by going over their monthly limit and paying extra money.
    7. In order to analyze customer behavior, average minutes, messages, internet and fee for users of each plan obtained.
    8. Several plots & Histograms were made to get an insight into the data:
        1.**"Calls" and "Internet" histograms have "Normal Distribution" type.**
        2. **"Message" histograms have "Right Skewed Distribution" type.**
3. **Step 4 conclusion:**
    1. **Test hypotheses no 1:**
        1. I set the alpha - the critical statistical significance level, at 0.05 (5%).
        2. I assumed that the null hypothesis would be rejected, alpha equal to 5% would have sufficed.
        3. I could use lower alpha to avoid a type 1 error and getting false positive results, but I decide to go first on the safe way and check the results, the result go as plan and the 5% was used.
        4. data_plans_fees, which contains average minutes, fee each user paid during analysis period, was used to test the hypothesis.
        5. With **95% confidence,** it was observed that revenue coming from each plan is **not equal.** More precisely, on average surf plan, users pay **USD 54.76 per month,** while ultimate plan users pay **USD72.33 per month.** However, the number of ultimate plan users are almost two times **less** than surf plan users.
        6. As we can see, even though there is a difference between average revenue generated by the users of each plan, **the p-value is lower than the alpha.** 
        7. **The null hypotesis was rejected.**
    2.  **Test hypotheses no 2:**
        1. I set the alpha - the critical statistical significance level, at 0.05 (5%).
        2. This means that the p-value (the probability value) is the probability of getting a result that the null hypothesis is correct - i.e., that there is a difference between the average revenue generated by the users in each group of regions - has to be greater than the alpha for us not to reject the said null hypothesis, and vice versa.
        3. "users" which contains the city, & fee each user paid during analysis period, was used to test the hypothesis.
        4. As we can see, even though there is little difference between average revenue generated by the users of each group of region, **the p-value is higher than the alpha.**
        5. **The null hypotesis can not be rejected**
4. **Bottom line conclusion:**
    1. **The company should invest more into an advertisement of the ultimate plan and attract more customers for the more profitable plan.**
    2. **The revenue coming from users in NY-NJ was not significantly different from other regions, For now, it is not important to focus on customers from NY-NJ, specifically.**


**THANK YOU FOR TAKING THE TIME TO REVIEW MY PROJECT and give me comments, point view and tips on how to improve my work. - Y.T**